# Imports

In [163]:
import torch 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import os
import json
import gensim
import re
import torch.optim as optim
from torchtext.vocab import GloVe
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score, f1_score
from gensim.models import KeyedVectors
from tqdm.notebook import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [186]:
with open('../processed/LR_train_tagged.json') as f: 
    LR_train = json.load(f)
    
with open('../processed/LR_test_tagged.json') as f:
    LR_test = json.load(f)
    
with open('../processed/LR_val_tagged.json') as f:
    LR_val = json.load(f)

with open('../processed/NER_train_tagged.json') as f:
    NER_train = json.load(f)

with open('../processed/NER_test_tagged.json') as f:
    NER_test = json.load(f)

with open('../processed/NER_val_tagged.json') as f:
    NER_val = json.load(f)


In [168]:
w2vmodel = KeyedVectors.load_word2vec_format('../.vector_cache/GoogleNews-vectors-negative300.bin', binary=True)
# glovemodel = GloVe(name='6B', dim=300)

In [169]:
unique_labels = []
for entry in NER_train:
    labels = entry['labels']
    for label in labels:
        unique_labels.append(label)

label_dict = {}
unique_labels = list(set(unique_labels))

for label_index in range(len(unique_labels)):
    label_dict[unique_labels[label_index]] = label_index

label_dict['PAD'] = len(unique_labels)
# label_dict['UNK'] = len(unique_labels) + 1
print(label_dict)

{'B_PRECEDENT': 0, 'I_PROVISION': 1, 'B_PROVISION': 2, 'B_ORG': 3, 'I_STATUTE': 4, 'B_DATE': 5, 'I_WITNESS': 6, 'I_DATE': 7, 'B_STATUTE': 8, 'I_PRECEDENT': 9, 'I_PETITIONER': 10, 'B_RESPONDENT': 11, 'I_ORG': 12, 'B_CASE_NUMBER': 13, 'I_OTHER_PERSON': 14, 'B_GPE': 15, 'B_COURT': 16, 'O': 17, 'B_WITNESS': 18, 'I_COURT': 19, 'I_CASE_NUMBER': 20, 'I_GPE': 21, 'I_JUDGE': 22, 'I_RESPONDENT': 23, 'B_JUDGE': 24, 'B_PETITIONER': 25, 'B_OTHER_PERSON': 26, 'PAD': 27}


In [505]:
maxlen = 0
id = 0
for entry in NER_train:
    if (maxlen < len(entry['text'].split())):
        maxlen = len(entry['text'].split())
        id = entry['id']
print(maxlen)
print(id)

70
dc09913bba844e3c8d920c9df2970988


In [199]:
class NER_dataset(Dataset):
    def __init__(self, dataset, embedding="word2vec"):
        self.padding_word = "PAD"
        if embedding == "word2vec":
            self.model = w2vmodel
        # elif embedding == "glove":
        #     self.model = glovemodel
        self.data = dataset
        self.input = []
        self.labels = []
        for entry in dataset:
            padded_entry = entry["text"].split() + [self.padding_word] * (75- len(entry["text"].split()))
            self.input.append(padded_entry)
            padded_labels = entry["labels"] + [self.padding_word] * (75 - len(entry["labels"]))
            self.labels.append(padded_labels)

            # self.input.append(entry["text"].split())
            # self.labels.append(entry["labels"])

        self.tag_to_index = label_dict

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        sentence = self.input[idx]
        word_vecs = []
        labels = []
        for word_index in range(len(sentence)):
            if sentence[word_index] not in self.model:
                # if self.labels[idx][word_index] != "O":
                #     print(sentence[word_index])
                word_vecs.append(np.zeros(300))
                labels.append(self.tag_to_index["O"])
                continue
            else:
                word_vecs.append(self.model[sentence[word_index]])
                labels.append(self.tag_to_index[self.labels[idx][word_index]])

        return torch.tensor(word_vecs), torch.tensor(labels)

In [200]:
NER_train_loader = DataLoader(NER_dataset(NER_train, "word2vec"), batch_size=64, shuffle=True)
NER_val_loader = DataLoader(NER_dataset(NER_val, "word2vec"), batch_size=64, shuffle=True)
NER_test_loader = DataLoader(NER_dataset(NER_test, "word2vec"), batch_size=64, shuffle=True)

In [201]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):

    train_losses = []
    val_losses = []
    train_f1_scores = []
    val_f1_scores = []
    train_accuracy_scores = []
    val_accuracy_scores = []

    for epoch in range(num_epochs):
        train_loss = 0.0
        val_loss = 0.0
        all_train_labels = []
        all_train_preds = []
        all_val_labels = []
        all_val_preds = []

        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device).to(torch.float32), labels.to(device).to(torch.long)
            outputs, hn = model(inputs)
            # print("old shape: ", outputs.shape)
            outputs = outputs.reshape(-1, outputs.shape[-1])
            # print("new shape: ", outputs.shape)
            labels = labels.reshape(-1)
            # print("labels shape: ", labels.shape)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            all_train_labels.extend(labels.cpu().numpy())
            all_train_preds.extend(predicted.cpu().numpy())

        train_accuracy = accuracy_score(all_train_labels, all_train_preds)
        train_f1 = f1_score(all_train_labels, all_train_preds, average='macro')
        train_accuracy_scores.append(train_accuracy)
        train_f1_scores.append(train_f1)

        model.eval()
        with torch.no_grad():

            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device).to(torch.float32), labels.to(device).to(torch.long)
                outputs, hn = model(inputs)
                outputs = outputs.reshape(-1, outputs.shape[-1])
                labels = labels.reshape(-1)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                all_val_labels.extend(labels.cpu().numpy())
                all_val_preds.extend(predicted.cpu().numpy())

            val_accuracy = accuracy_score(all_val_labels, all_val_preds)
            val_f1 = f1_score(all_val_labels, all_val_preds, average='macro')
            val_accuracy_scores.append(val_accuracy)
            val_f1_scores.append(val_f1)

        train_losses.append(train_loss/len(train_loader))
        val_losses.append(val_loss/len(val_loader))
        print(f"Epoch {epoch + 1}\n"
              f"Train loss: {train_losses[-1]}, Val loss: {val_losses[-1]}\n"
              f"Train accuracy: {train_accuracy}, Val accuracy: {val_accuracy}\n"
              f"Train F1: {train_f1}, Val F1: {val_f1}")
        print("=====================================================================================================")
        

In [137]:
class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(300, 128, batch_first=True)
        self.linear = nn.Linear(128, len(label_dict))
        self.relu = nn.ReLU()

    def forward(self, x):
        x, hn = self.rnn(x)
        x = self.relu(x)
        x = self.linear(x)
        return x, hn

In [202]:
# model = torch.nn.RNN(300, len(label_dict), 2, batch_first=True)
model = torch.nn.LSTM(300, len(label_dict), 1, batch_first=True)
# model = torch.nn.GRU(300, len(label_dict), 1, batch_first=True)
# model = RNNModel()
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [203]:
train(model, NER_train_loader, NER_val_loader, criterion, optimizer, num_epochs=5)

Epoch 1
Train loss: 1.774511386477758, Val loss: 1.6592756924421892
Train accuracy: 0.9235748430810159, Val accuracy: 0.949322033898305
Train F1: 0.0693409157237286, Val F1: 0.06990946199517091
Epoch 2
Train loss: 1.65402147599629, Val loss: 1.653259945952374
Train accuracy: 0.95132393897826, Val accuracy: 0.9496516007532957
Train F1: 0.06955307304424747, Val F1: 0.06943986353676837


KeyboardInterrupt: 

In [140]:
model.eval()
all_test_labels = []
all_test_preds = []
with torch.no_grad():
    for inputs, labels in NER_test_loader:
        inputs, labels = inputs.to(device).to(torch.float32), labels.to(device).to(torch.long)
        outputs, hn = model(inputs)
        outputs = outputs.reshape(-1, outputs.shape[-1])
        labels = labels.reshape(-1)

        _, predicted = torch.max(outputs, 1)
        all_test_labels.extend(labels.cpu().numpy())
        all_test_preds.extend(predicted.cpu().numpy())

test_accuracy = accuracy_score(all_test_labels, all_test_preds)
test_f1 = f1_score(all_test_labels, all_test_preds, average='macro')
print(f"Test accuracy: {test_accuracy} Test F1: {test_f1}")

Test accuracy: 0.9632465277777778 Test F1: 0.2976833335124903


In [55]:
for input, label, sentence in NER_train_loader:
    print(sentence)
    outputs, hn = model(input.to(device).to(torch.float32))
    # print label and prediction
    _, predicted = torch.max(outputs, 2)
    print(label)
    print(predicted)

    print(outputs.shape)
    break
    

('The injuries of Shish Pal (D-3) are as follows: (i) Incised Wounds at (Lt) pareital area of skull at top size 12 cm x 1-1/2 cms x Brain matter deep obliquely & Tailing Backwards. (ii) Multiple Incised Wounds at (Lt) Head Laterally & at Back side. ', 'The learned Counsel for the parties informed that the matter is still not decided by the larger Bench of the Supreme Court. ', 'therefore, she was not entitled to any damages. ', 'Some of those 15 also had injuries on their persons which appeared to be due to lathi charge at village Khabra. ', 'I am unable to subscribe to the Second reasoning given by Piggott J. in AIR 1916 All 51 that if an appeal against a decree can lie in certain circumstances the decree may be regarded as one open to appeal for purposes of Order 43 Rule 1(d). ', 'According to Sri Haranahalli, the complaint as against respondent/accused No. 2 was definitely maintainable and could be proceeded with against him in view of no bar thereto under section 446 of the Compani